## Lab 7 :Document Clustering


Document clustering yet again includes similar steps, so let’s have a look at
them:
1. Tokenization
2. Stemming and lemmatization
3. Removing stop words and punctuation
4. Computing term frequencies or TF-IDF
5. Clustering: K-means/Hierarchical; we can then use
any of the clustering algorithms to cluster different
documents based on the features we have generated
6. Evaluation and visualization: Finally, the clustering
results can be visualized by plotting the clusters into
a two-dimensional space

### 1)Install and import needed libraries:

In [ ]:
#!pip install mpld3
#! pip install bs4
import numpy as np
import pandas as pd
import nltk
from nltk.stem.snowball import SnowballStemmer
from bs4 import BeautifulSoup
import re
import os
import codecs # for encoding and decoding
from sklearn import feature_extraction
import mpld3  #The mpld3 project brings together Matplotlib, the popular Python-based graphing library, and D3js, the popular JavaScript library for creating interactive data visualizations for the web. The result is a simple API for exporting your matplotlib graphics to HTML code which can be used within the browser, within standard web pages, blogs, or tools such as the IPython notebook.
from sklearn.metrics.pairwise import cosine_similarity  
import os 
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.manifold import MDS


### 2)Read Data set:

In [ ]:
#Read Data set
Data = pd.read_csv("Consumer_Complaints.csv",encoding='latin-1')

#selecting required columns and rows
Data = Data[['consumer_complaint_narrative']]
Data = Data[pd.notnull(Data['consumer_complaint_narrative'])]

# Take a sample of just 200 documents. 
Data_sample=Data.sample(200)

C:\Users\Mona\anaconda3\envs\tfcpu\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (5,11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


### 3)Data Preprocessing:

In [ ]:
Data_sample['consumer_complaint_narrative'] = Data_sample['consumer_complaint_narrative'].str.replace('XXXX','')

# Convert dataframe to list
complaints = Data_sample['consumer_complaint_narrative'].tolist()

# create the rank of documents – we will use it later

ranks = []
for i in range(1, len(complaints)+1):
    ranks.append(i)

# Stop Words
stopwords = nltk.corpus.stopwords.words('english')

# Load 'stemmer'
stemmer = SnowballStemmer("english")

# Functions for sentence tokenizer, to remove numeric tokens and raw #punctuation

def tokenize_and_stem(text):
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

def tokenize_only(text):
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

from sklearn.feature_extraction.text import TfidfVectorizer

# tfidf vectorizer 

tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=200000,
                                 min_df=0.2, stop_words='english',
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

#fit the vectorizer to data

tfidf_matrix = tfidf_vectorizer.fit_transform(complaints) 
terms = tfidf_vectorizer.get_feature_names()
print(tfidf_matrix.shape)


C:\Users\Mona\anaconda3\envs\tfcpu\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'mani', 'meanwhil', 'moreov', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'togeth', 'twelv', 'twenti', 'veri', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


(200, 29)


### 4) k-means Clustering :

In [ ]:
#Import Kmeans
from sklearn.cluster import KMeans

# Define number of clusters
num_clusters = 6

#Running clustering algorithm
km = KMeans(n_clusters=num_clusters)
km.fit(tfidf_matrix)

#final clusters
clusters = km.labels_.tolist()
complaints_data = { 'rank': ranks, 'complaints': complaints, 'cluster': clusters }
frame = pd.DataFrame(complaints_data, index = [clusters] , columns = ['rank', 'cluster'])

#number of docs per cluster 
frame['cluster'].value_counts()

1    50
4    37
5    33
0    29
2    28
3    23
Name: cluster, dtype: int64

In [ ]:
totalvocab_stemmed = []
totalvocab_tokenized = []
for i in complaints:
    allwords_stemmed = tokenize_and_stem(i) 
    totalvocab_stemmed.extend(allwords_stemmed) 
    
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)

vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)

#sort cluster centers by proximity to centroid
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

for i in range(num_clusters):
    print("Cluster %d words:" % i, end='')
    
    for ind in order_centroids[i, :6]: 
        print(' %s' % vocab_frame.loc[terms[ind].split(' ')].values.tolist()[0][0].encode('utf-8', 'ignore'), end=',')
    print()

Cluster 0 words: b'loan', b'paying', b'because', b'company', b"n't", b'payments',
Cluster 1 words: b'reports', b'credit', b'credit', b'information', b'account', b'year',
Cluster 2 words: b'payments', b'month', b'told', b'paying', b'services', b'receiving',
Cluster 3 words: b'debt', b'account', b'company', b'credit', b'services', b'asked',
Cluster 4 words: b'state', b'information', b'company', b"n't", b'any', b'contacted',
Cluster 5 words: b'account', b'bank', b'day', b'credit', b'time', b'used',


### 5) Data Visualization:

In [ ]:
#Similarity

similarity_distance = 1 - cosine_similarity(tfidf_matrix)


# Convert two components as we're plotting points in a two-dimensional plane
mds = MDS(n_components=2, dissimilarity="precomputed", random_state=1)
pos = mds.fit_transform(similarity_distance)  # shape (n_components, n_samples)
xs, ys = pos[:, 0], pos[:, 1]

#Set up colors per clusters using a dict
cluster_colors = {0: '#1b9e77', 1: '#d95f02', 2: '#7570b3', 3: '#e7298a', 4: '#66a61e',5: '#D2691E'}

#set up cluster names using a dict
cluster_names = {0: 'property, based, assist', 
                 1: 'business, card', 
                 2: 'authorized, approved, believe', 
                 3: 'agreement, application,business', 
                 4: 'closed, applied, additional',
                 5: 'applied, card'}

# Finally plot it
%matplotlib inline 

#Create data frame that has the result of the MDS and the cluster 
df = pd.DataFrame(dict(x=xs, y=ys, label=clusters)) 
groups = df.groupby('label')

# Set up plot
fig, ax = plt.subplots(figsize=(17, 9)) # set size

for name, group in groups:
    ax.plot(group.x, group.y, marker='o', linestyle='', ms=20, 
            label=cluster_names[name], color=cluster_colors[name], 
            mec='none')
    ax.set_aspect('auto')
    ax.tick_params(\
        axis= 'x',          
        which='both',      
        bottom='off',     
        top='off',         
        labelbottom='off')
    ax.tick_params(\
        axis= 'y',        
        which='both',    
        left='off',     
        top='off',       
        labelleft='off')
    
ax.legend(numpoints=1) 
plt.show()
